# Generazione della griglia

Viene creata una matrice ```g``` di 1 con ```m``` righe e ```n``` colonne, il parametro ```density``` controlla il numero di celle attraversabili e quindi il numero di ostacoli, mentre  ```agglomerate``` definisce la dimensione di un agglomerato. 

In [669]:
import numpy as np
import random

In [670]:
nrow = 10 
ncol = 15
g =  (np.zeros((nrow,ncol))).astype(int)
density = 0.8
agglomerate = 3

if agglomerate > nrow | agglomerate > ncol :
    print("Agglomerato troppo grande")

print(g)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


vengono calcolati il numero di ostacoli usando ```density```, successivamente vengono calcolati il numero di agglomerati facendo la divisione intera tra ostacoli e ```agglomerate```, infine viene calcolata la dimensione dell'ultimo agglomerato

In [671]:
n_obstacle = nrow * ncol * (1 - density)
n_agglomerates = n_obstacle / agglomerate
last_agglomerate = n_obstacle % agglomerate

Vengono istanziati due dizionari, uno per le righe e uno per le colonne che serviranno per l'inserimento degli agglomerati. Il primo dizionario avrà le chiavi che saranno le righe della matrice ```g```, mentre i valori sono le liste degli intervalli delle celle libere della riga. Stessa cosa per l'altro dizionario ma fatto per le colonne 

In [672]:
range_row = { i : [(0, ncol-1)] for i in range(0,nrow) }
range_col = { j : [(0, nrow-1)] for j in range(0,ncol) }

print(range_row)
print(range_col)


{0: [(0, 14)], 1: [(0, 14)], 2: [(0, 14)], 3: [(0, 14)], 4: [(0, 14)], 5: [(0, 14)], 6: [(0, 14)], 7: [(0, 14)], 8: [(0, 14)], 9: [(0, 14)]}
{0: [(0, 9)], 1: [(0, 9)], 2: [(0, 9)], 3: [(0, 9)], 4: [(0, 9)], 5: [(0, 9)], 6: [(0, 9)], 7: [(0, 9)], 8: [(0, 9)], 9: [(0, 9)], 10: [(0, 9)], 11: [(0, 9)], 12: [(0, 9)], 13: [(0, 9)], 14: [(0, 9)]}


La funzione ```add_agglomerate``` riceve in parametro la matrice ```g```, la dimensione dell'agglomerato e i due dizionari. Il flag ```controlled``` è riferito alla sovrapposizione di agglomerati

In [673]:
def add_agglomerate(g, agglomerate, range_row, range_col, controlled = True):
    print("----- Add agglomerate -----")
    if len(range_row.keys()) <1 and len(range_col.keys())<1:
        print("no space for add agglomerate")
        return
    axis = 0 if random.random() < 0.5 else 1
    if len(range_row.keys()) < 1:
        axis = 1
    if len(range_col.keys()) < 1:
        axis = 0
    print("axis : " +  str(axis))
    range_dict = range_row if axis == 0 else range_col
    other_dict = range_col if axis == 0 else range_row


    # extract row/col and one of its ranges
    line = random.choice(list(range_dict.keys()))
    print("random line selected : " + str(line))
    range_line = random.choice(range_dict[line])
    print("random range selected : " + str(range_line))

    # create agglomerate
    start_agg = random.randrange(range_line[0], range_line[1] + 2 - agglomerate)
    end_agg = start_agg + agglomerate -1
    print("agglomerate :" + str(start_agg) + " - " + str(end_agg))

    # update matrix g
    if axis == 0:
        for i in range(start_agg, end_agg + 1): g[line][i] = 1 
    else :
        for i in range(start_agg, end_agg + 1): g[i][line] = 1

    
    if controlled :
        print("---- update dictonaries ----")
        # modify range_line in the range_dict 
        print("--- update range_dict ---")
        range_dict[line].remove(range_line)    
        if start_agg - range_line[0] >= agglomerate :
            range_dict[line].append((range_line[0], start_agg - 1))
            print("add lower range : " + str(range_line[0]) + " - " + str(start_agg - 1))
        if range_line[1] - end_agg >= agglomerate :
            range_dict[line].append((end_agg+1, range_line[1]))
            print("add upper range : " + str(end_agg + 1) + " - " + str(range_line[1]))
        if len(range_dict[line]) == 0:
            range_dict.pop(line)
            print("no range greater than agglomerate in line " + str(line))
        
        print("--- update other_dict ---")
        # modify all the range in the other_dict
        for other_line in range(start_agg, end_agg+1):
            if other_line not in other_dict.keys():
                print("skipped line " + str(other_line))
                continue
            other_ranges = other_dict[other_line]
            for r in other_ranges:
                if r[0] <= line and r[1] >= line:
                    print("in other_line " + str(other_line) + " remove " + str(r))
                    other_ranges.remove(r)
                    if line - r[0] >= agglomerate :
                        new_range = (r[0], line-1)
                        other_ranges.append(new_range)
                        print("added lower other_range : " + str(new_range))

                    if r[1] - line >= agglomerate :
                        new_range = (line +1, r[1])
                        other_ranges.append(new_range)
                        print("added upper to other_range : " + str(new_range))
            if len(other_ranges) == 0:
                print("no range greater than agglomerate in other_line " + str(other_line))
                other_dict.pop(other_line)

while n_agglomerates> 0 :
    add_agglomerate(g, agglomerate, range_row, range_col)
    n_agglomerates= n_agglomerates-1
print(g)
print("range row : " +str(range_row))
print("range col : " + str(range_col))

----- Add agglomerate -----
axis : 0
random line selected : 6
random range selected : (0, 14)
agglomerate :1 - 3
---- update dictonaries ----
--- update range_dict ---
add upper range : 4 - 14
--- update other_dict ---
in other_line 1 remove (0, 9)
added lower other_range : (0, 5)
added upper to other_range : (7, 9)
in other_line 2 remove (0, 9)
added lower other_range : (0, 5)
added upper to other_range : (7, 9)
in other_line 3 remove (0, 9)
added lower other_range : (0, 5)
added upper to other_range : (7, 9)
----- Add agglomerate -----
axis : 0
random line selected : 4
random range selected : (0, 14)
agglomerate :3 - 5
---- update dictonaries ----
--- update range_dict ---
add lower range : 0 - 2
add upper range : 6 - 14
--- update other_dict ---
in other_line 3 remove (0, 5)
added lower other_range : (0, 3)
in other_line 4 remove (0, 9)
added lower other_range : (0, 3)
added upper to other_range : (5, 9)
in other_line 5 remove (0, 9)
added lower other_range : (0, 3)
added upper to o

In [680]:

arr = np.frombuffer(str(g), count= 15)

print(arr)

TypeError: a bytes-like object is required, not 'str'